## Using FNET : [Paper_Link](https://arxiv.org/abs/2105.03824)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        break
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir('/kaggle/input/petfinder-pawpularity-score')

In [ ]:
IMG_PATH = '/kaggle/input/petfinder-pawpularity-score/train/'

In [ ]:
df_train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
import io
import shutil, random
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
from PIL import Image
from pathlib import Path
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from sklearn.utils import shuffle
from tensorflow.keras import layers
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
weight_decay = 0.0001
batch_size = 64
num_epochs = 5
dropout_rate = 0.2
image_size = 224  # We'll resize input images to this size.
patch_size = 8  # Size of the patches to be extracted from the input images.
num_patches = (image_size // patch_size) ** 2  # Size of the data array.
embedding_dim = 256  # Number of hidden units.
num_blocks = 4  # Number of blocks.
num_classes = 1

In [ ]:
image_data = []
labels = []

for idx in range(df_train.shape[0]):
#     print(df_train['Id'][idx], df_train['Pawpularity'][idx])
    img_path = IMG_PATH + str(df_train['Id'][idx]) + '.jpg'
    try:
        img = tf.keras.preprocessing.image.load_img(img_path, color_mode='rgb', target_size= (image_size, image_size))
        img = np.array(img)
        image_data.append(img)
        labels.append(df_train['Pawpularity'][idx])
    except:
        ...
    else:
        ...

In [ ]:
print(len(image_data),len(labels))

In [ ]:
combined = list(zip(image_data,labels))
random.shuffle(combined)

In [ ]:
X_train = np.array(image_data)
Y_train = np.array(labels)
Y_train = Y_train.reshape((Y_train.shape[0], 1))
Y_train = Y_train.astype("float32")
print(X_train.shape)
print(Y_train.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
y_train.dtype

In [ ]:
"""
## Use data augmentation
"""

data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)

In [ ]:
"""
## Build a regression model
We implement a method that builds a regression model given the processing blocks.
"""


def build_classifier(blocks, positional_encoding=False):
    inputs = layers.Input(shape=(image_size, image_size, 3))
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size, num_patches)(augmented)
    # Encode patches to generate a [batch_size, num_patches, embedding_dim] tensor.
    x = layers.Dense(units=embedding_dim)(patches)
    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding
    # Process x using the module blocks.
    x = blocks(x)
    # Apply global average pooling to generate a [batch_size, embedding_dim] representation tensor.
    representation = layers.GlobalAveragePooling1D()(x)
    # Apply dropout.
    representation = layers.Dropout(rate=dropout_rate)(representation)
    # Compute logits outputs.
    logits = layers.Dense(num_classes, activation = 'linear')(representation)
    # Create the Keras model.
    return keras.Model(inputs=inputs, outputs=logits)

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
"""
## Define an experiment
We implement a utility function to compile, train, and evaluate a given model.
"""


def run_experiment(model):
    # Create Adam optimizer with weight decay.
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay,
    )
    
    # Compile the model.
    model.compile(
        optimizer = optimizer,
        loss = root_mean_squared_error,
        metrics=[
            keras.metrics.RootMeanSquaredError()
        ],
    )
    
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
    )
    
    print(model.evaluate(x_test, y_test))

    # Return history to plot learning curves.
    return history
    
#     _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")
#     print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
    
    

In [ ]:
"""
## Implement patch extraction as a layer
"""


class Patches(layers.Layer):
    def __init__(self, patch_size, num_patches):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        self.num_patches = num_patches

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches

In [ ]:
"""
## The FNet model
The FNet uses a similar block to the Transformer block. However, FNet replaces the self-attention layer
in the Transformer block with a parameter-free 2D Fourier transformation layer:
1. One 1D Fourier Transform is applied along the patches.
2. One 1D Fourier Transform is applied along the channels.
"""

"""
### Implement the FNet module
"""

class FNetLayer(layers.Layer):
    def __init__(self, num_patches, embedding_dim, dropout_rate, *args, **kwargs):
        super(FNetLayer, self).__init__(*args, **kwargs)

        self.ffn = keras.Sequential(
            [
                layers.Dense(units=embedding_dim),
                tfa.layers.GELU(),
                layers.Dropout(rate=dropout_rate),
                layers.Dense(units=embedding_dim),
            ]
        )

        self.normalize1 = layers.LayerNormalization(epsilon=1e-6)
        self.normalize2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply fourier transformations.
        x = tf.cast(
            tf.signal.fft2d(tf.cast(inputs, dtype=tf.dtypes.complex64)),
            dtype=tf.dtypes.float32,
        )
        # Add skip connection.
        x = x + inputs
        # Apply layer normalization.
        x = self.normalize1(x)
        # Apply Feedfowrad network.
        x_ffn = self.ffn(x)
        # Add skip connection.
        x = x + x_ffn
        # Apply layer normalization.
        return self.normalize2(x)

In [ ]:
"""
### Build, train, and evaluate the FNet model
"""

fnet_blocks = keras.Sequential(
    [FNetLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.001
fnet_classifier = build_classifier(fnet_blocks, positional_encoding=True)
history = run_experiment(fnet_classifier)

In [ ]:
df_test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

In [ ]:
df_test.head()

In [ ]:
sample_sub = df_test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
sample_sub.head()

In [ ]:
TEST_IMG_PATH = '/kaggle/input/petfinder-pawpularity-score/test/'

In [ ]:
output = {
    'Id' : [],
    'Pawpularity' : []
}

for idx in range(df_test.shape[0]):
#     print(df_train['Id'][idx], df_train['Pawpularity'][idx])
    img_path = TEST_IMG_PATH + str(df_test['Id'][idx]) + '.jpg'
    try:
        image = tf.keras.preprocessing.image.load_img(img_path, color_mode='rgb', target_size= (image_size, image_size))
        image = np.array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        preds = fnet_classifier.predict(image)
        output['Id'].append(df_test['Id'][idx])
        output['Pawpularity'].append(preds[0][0])
#         print(preds[0][0])
    except:
        ...
    else:
        ...

In [ ]:
sub = pd.DataFrame(output)

In [ ]:
sub.shape

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index = False)